In [0]:
# !pip install -q tensorflow-gpu==2.0.0-alpha0
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import sklearn
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv1D

In [0]:
f = "audpred2.csv"

df = pd.read_csv(f)

df = pd.read_csv(f)

In [7]:
file_name = "wma3"
# x = df[["Close","wma","ema","sma"]]
x = df[[file_name]]
y = df[[file_name]]

fit = preprocessing.MinMaxScaler().fit(y)
x = preprocessing.MinMaxScaler().fit_transform(x)
y = fit.transform(y)

num_nits = 24
s = 1
gen1 = tf.keras.preprocessing.sequence.TimeseriesGenerator(x,x,num_nits,stride=1, batch_size=3000)
x = []
y = []
for i in range(len(gen1)):
  xx,yy = gen1[i]
  xx,yy = xx.tolist(),yy.tolist()
  x.extend(xx)
  y.extend(yy)

x = np.asanyarray(x)
y = np.asanyarray(y).reshape((-1,s))

train_x,test_x,train_y,test_y = train_test_split(x,y,test_size=0.3,  shuffle =True)
y.shape

(87494, 1)

In [8]:
def model():
    def block(inputs,type=1):
        cnn1 = layers.Conv1D(4,2,padding="causal",activation="relu")(inputs)
        cnn2 = layers.Conv1D(8,2,padding="causal",activation="relu")(inputs)
        cnn3 = layers.Conv1D(16,2,padding="causal",activation="relu")(inputs)
        cnn4 = layers.Conv1D(32,2,padding="causal",activation="relu")(inputs)
        cnn5 = layers.Conv1D(64,2,padding="causal",activation="relu")(inputs)

        concat =  layers.Concatenate()([cnn1,cnn2,cnn3,cnn4,cnn5])

        if type  == 1:
            concat = layers.ReLU()(concat)
        return concat

    inputs = layers.Input((num_nits,1))

    block1 = block(inputs)
    block1 = block(inputs,2)

    block2 = block(block1)
    block2 = block(block1,2)
    block3 = block(block1)
    block3 = block(block1,2)

    add = layers.Add()([block1,block2,block3])

    cnn = layers.Conv1D(128,2,padding="causal",activation="relu")(add)

    flatten = layers.Flatten()(cnn)
    outputs = layers.Dense(1)(flatten)

    model = tf.keras.Model(inputs,outputs)
    tf.keras.utils.plot_model(model)

    return model
  
model = model()

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
# model = tf.keras.models.load_model("close2.hdf5")

In [0]:
batch_size = 10000

In [11]:
model.compile(tf.keras.optimizers.Adam(0.001),"mse",["mae"])

Instructions for updating:
Use tf.cast instead.


In [0]:
model.fit(train_x,train_y,batch_size,4000, validation_data=[test_x,test_y])

61245/61245 [==============================] - 1s 20us/sample - loss: 2.5955e-06 - mean_absolute_error: 0.0013 - val_loss: 1.3928e-06 - val_mean_absolute_error: 8.4294e-04
Epoch 1445/4000
61245/61245 [==============================] - 1s 20us/sample - loss: 1.2781e-06 - mean_absolute_error: 7.6949e-04 - val_loss: 1.1775e-06 - val_mean_absolute_error: 7.2796e-04
Epoch 1446/4000
61245/61245 [==============================] - 1s 20us/sample - loss: 1.2486e-06 - mean_absolute_error: 7.6448e-04 - val_loss: 2.1864e-06 - val_mean_absolute_error: 0.0012
Epoch 1447/4000
61245/61245 [==============================] - 1s 20us/sample - loss: 1.0330e-05 - mean_absolute_error: 0.0027 - val_loss: 5.9754e-05 - val_mean_absolute_error: 0.0075
Epoch 1448/4000
61245/61245 [==============================] - 1s 20us/sample - loss: 5.7580e-05 - mean_absolute_error: 0.0069 - val_loss: 1.7980e-05 - val_mean_absolute_error: 0.0038
Epoch 1449/4000
61245/61245 [==============================] - 1s 20us/sample - 

In [0]:
model.evaluate(x,y,batch_size,use_multiprocessing=True)

In [0]:
model.compile(tf.keras.optimizers.Adam(0.0001),"mse",["mae"])

In [0]:
model.fit(train_x,train_y,batch_size,3000, validation_data=[test_x,test_y])

In [0]:
model.evaluate(x,y,batch_size,use_multiprocessing=True)

In [0]:
model.compile(tf.keras.optimizers.Adam(0.00001),"mse",["mae"])

In [0]:
model.fit(train_x,train_y,batch_size,3000, validation_data=[test_x,test_y])

In [0]:
import matplotlib.pyplot as plt

pred = model.predict(x,3000)

# y = y[0-1]
# pred = pred[1::]

# print(((pred-y)**2).mean())
y3 = y[-100::]
y4 = pred[-100::]
# print(((y4-y3)**2).mean())


plt.plot(y3)
plt.plot(y4)
plt.title('accuracy')
plt.ylabel('test')
plt.show()
  
plt.plot(y4)
plt.title('accuracy')
plt.ylabel('test')
plt.show()

plt.plot(y3)
plt.title('accuracy')
plt.ylabel('test')
plt.show()
7.61011205413085e-10

In [0]:
ud = []
pre_ud = []
# pre_ud2 = []

for i in range(1,len(y)):
  ud.extend(np.where(y[i]>y[i-1] ,"buy","sell"))
  pre_ud.extend(np.where(pred[i]>y[i-1],"buy","sell"))
#   pre_ud2.extend(np.where(pred[i]>pred[i-1],"buy","sell"))


# cp = cp[0:-1]
print(sklearn.metrics.accuracy_score(ud,pre_ud))

cpp = np.asanyarray(df[["Open"]])
ud = ud[-1900::]
pre_ud = pre_ud[-1900::]
# pre_ud2 = pre_ud2[-1900::]
cpp = cpp[-1900::]
pre_ud2 = pre_ud
print(sklearn.metrics.accuracy_score(ud,pre_ud2))

In [0]:
np.mean(ud==pre_ud)

In [0]:
# pips = 100000
pips = 1000
pip = []
trade = []
buy = 0
sell = 0
clear = 0
close = "False"
tp = 10000
loscut = -20

spread = 0.01

# pre_ud = ud
pre_ud = pre_ud2

for i in range(0,len(pre_ud)):
    if len(trade) == 0:
        if pre_ud[i] == "buy":
          sell = np.float(cpp[i]) - spread
          trade.append("buy")
        if ud == "sell":
          buy = np.float(cpp[i]) + spread
          trade.append("sell")
    
    else:
        if pre_ud[i] == "buy" and trade[-1] == "buy":
            clear = np.float(cpp[i])
            Pip = (sell - clear)*pips
            if Pip < loscut and close == "False":
#                 print(Pip)
                Pip = loscut
                close = "True"
                pip.append(Pip)
            elif Pip > tp and close == "False":
                Pip = tp
                close = "True"
                pip.append(Pip)
            trade.append("buy")
        
        elif pre_ud[i] == "buy" and trade[-1] == "sell":
            if close == "False":
                clear = np.float(cpp[i])
                Pip = (clear - buy)*pips
                if Pip < loscut:
                    Pip = loscut
                pip.append(Pip)
            else:
                close = "False"
            buy = 0
            sell = clear - spread
            trade.append("buy")

        elif pre_ud[i] == "sell" and trade[-1] == "sell":
            clear = np.float(cpp[i])
            Pip = (clear - buy)*pips
            if Pip < loscut and close == "False":
                Pip = loscut
                close = "True"
                pip.append(Pip)
            elif Pip > tp and close == "False":
                Pip = tp
                close = "True"
                pip.append(Pip)
            trade.append("sell")

        elif pre_ud[i] == "sell" and trade[-1] == "buy":
            if close == "False":
                clear = np.float(cpp[i])
                Pip = (sell - clear)*pips
                if Pip < loscut:
                    Pip = loscut
                pip.append(Pip)
            else:
                close = "False"
            sell = 0
            buy = clear  + spread
            trade.append("sell")




print("pip の合計 : ",sum(pip))
print("trade回数 = ",len(pip))
print("max pip = ",max(pip))

print("")
print("総利益 : ", (sum(pip)/100)*10000)
print("")

print("accuracy : ",np.mean(np.asanyarray(pip)>=0))

print("")
print("")
print("                      利益 ヒストグラム")
print("")
print("")
plt.hist(pip,50)
plt.title('accuracy')
plt.ylabel('test')
plt.show()

In [0]:
# pips = 100000
pips = 1000
pip = []
trade = []
buy = 0
sell = 0
clear = 0
spread = 0.002
loscut = 10

# pre_ud = ud
pre_ud2 = pre_ud

for i in range(0,len(pre_ud)):
    if len(trade) == 0:
        if pre_ud[i] == "buy" and  pre_ud2[i] == "buy":
          buy = np.float(cpp[i]) - spread
          trade.append("buy")
        if pre_ud[i] == "sell" and  pre_ud2[i] == "sell":
          sell = np.float(cpp[i])+ spread
          trade.append("sell")
    
    else:
      if pre_ud[i] == "buy" and pre_ud2[i] == "buy":
        
        if trade[-1] == "buy":
          trade.append("buy")
        elif trade[-1] == "sell":
          clear = np.float(cpp[i])
          Pip = (sell - clear)*pips
          if Pip < -loscut:
            Pip = -loscut
#           Pip += -7
          pip.append(Pip)
          buy = clear + spread
          sell = 0
          trade.append("buy")
      elif pre_ud[i] == "sell" and pre_ud2[i] == "sell":
        if trade[-1] == "sell":      
          trade.append("sell")
        else:
          clear = np.float(cpp[i])
          
          Pip = (clear - buy)*pips
          if Pip < -loscut:
            Pip = -loscut
#           Pip += -7
          pip.append(Pip)
          sell = clear - spread
          buy = 0
          trade.append("sell")
          
print("pip の合計 : ",sum(pip))
print("trade回数 = ",len(pip))
print("max pip = ",max(pip))

print("")
print("総利益 : ", (sum(pip)/100)*10000)
print("")

print("accuracy : ",np.mean(np.asanyarray(pip)>=0))

print("")
print("")
print("                      利益 ヒストグラム")
print("")
print("")
plt.hist(pip,100)
plt.title('accuracy')
plt.ylabel('test')
plt.show()

In [0]:
model.save("wma.hdf5")

In [0]:
model = tf.keras.models.load_model("wma.hdf5")

In [0]:
for layer in model.layers:
    if layer == model.layers[-1]:
      pass
    else:
      layer.trainable = False